In [39]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

In [11]:
train_data = pd.read_csv("../train_data.csv")
dev_data = pd.read_csv("../dev_data.csv")

train_data = train_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
train_data["is_retweet"] = 1*train_data["is_retweet"]
train_data["is_modified_retweet"] = 1*train_data["is_modified_retweet"]

dev_data = dev_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
dev_data["is_retweet"] = 1*dev_data["is_retweet"]
dev_data["is_modified_retweet"] = 1*dev_data["is_modified_retweet"]

In [20]:
# X_train = train_data.drop(['tweet','author_Id'],axis=1).values
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# y_train = to_categorical(train_data["author_Id"])

sc = train_data.groupby("author_Id").count().sort_values("tweet",ascending = False)#.head()
set_y = list(sc.index)

In [23]:
def id_to_int_cat(id_ind):
    return np.argwhere(train_data["author_Id"].values[id_ind]==set_y)[0][0]
temp = [id_to_int_cat(i) for i in range(len(train_data["author_Id"]))]
y_train = to_categorical(temp,dtype=int)

In [25]:
X_train = train_data.drop(['tweet','author_Id'],axis=1).values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = to_categorical(temp,dtype=int)

In [30]:
def make_ff():
    model = Sequential()
    model.add(Dense(1000,activation = "relu",input_dim=X_train.shape[1],kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
    # model.add(Dense(5000,activation = "relu",kernel_regularizer=regularizers.l2(0.01)))
    # model.add(Dropout(0.3))
    model.add(Dense(y_train.shape[1],activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'],)
    
    return model


In [49]:
filepath = "modelgrid.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop =  EarlyStopping(monitor='val_loss',patience=3, min_delta=0.0001)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="log")
callbacks_list = [checkpoint,tensorboard_callback]#,early_stop,]


epochs = 3
batch_size = 128
model = make_ff()
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.25,
                    callbacks=callbacks_list)

Train on 185024 samples, validate on 61675 samples
Epoch 1/3
184960/185024 [============================>.] - ETA: 0s - loss: 8.9856 - accuracy: 0.0012
Epoch 00001: val_loss improved from inf to 8.92410, saving model to model.h5
185024/185024 [==============================] - 67s 360us/sample - loss: 8.9856 - accuracy: 0.0012 - val_loss: 8.9241 - val_accuracy: 0.0015
Epoch 2/3
184960/185024 [============================>.] - ETA: 0s - loss: 8.8745 - accuracy: 0.0017
Epoch 00002: val_loss improved from 8.92410 to 8.88608, saving model to model.h5
185024/185024 [==============================] - 65s 351us/sample - loss: 8.8744 - accuracy: 0.0017 - val_loss: 8.8861 - val_accuracy: 0.0019
Epoch 3/3
184832/185024 [============================>.] - ETA: 0s - loss: 8.8257 - accuracy: 0.0019
Epoch 00003: val_loss improved from 8.88608 to 8.84868, saving model to model.h5
185024/185024 [==============================] - 64s 347us/sample - loss: 8.8257 - accuracy: 0.0019 - val_loss: 8.8487 - va

In [ ]:
%tensorboard --logdir log

In [45]:
!pwd

/Users/maharaj/Documents/WhoTweeted/split_model


In [52]:
new_model = load_model("model.h5")
epochs = 100
batch_size = 64

history = new_model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.25,
                    callbacks=callbacks_list)

Train on 185024 samples, validate on 61675 samples
Epoch 1/100
184896/185024 [============================>.] - ETA: 0s - loss: 8.3266 - accuracy: 0.0074
Epoch 00001: val_loss did not improve from 8.53123
185024/185024 [==============================] - 86s 466us/sample - loss: 8.3267 - accuracy: 0.0074 - val_loss: 8.5324 - val_accuracy: 0.0073
Epoch 2/100
184960/185024 [============================>.] - ETA: 0s - loss: 8.3241 - accuracy: 0.0072
Epoch 00002: val_loss improved from 8.53123 to 8.52868, saving model to model.h5
185024/185024 [==============================] - 87s 468us/sample - loss: 8.3241 - accuracy: 0.0072 - val_loss: 8.5287 - val_accuracy: 0.0075
Epoch 3/100
184960/185024 [============================>.] - ETA: 0s - loss: 8.3202 - accuracy: 0.0073
Epoch 00003: val_loss did not improve from 8.52868
185024/185024 [==============================] - 90s 485us/sample - loss: 8.3202 - accuracy: 0.0073 - val_loss: 8.5329 - val_accuracy: 0.0071
Epoch 4/100
184960/185024 [====

KeyboardInterrupt: 